# Deep Learning-based for Vietnamese Text Classification

## CNN (Convolutional Neural Network)

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

repo = 'drive/MyDrive/Dive Into Code/Sprint 26/'

Mounted at /content/drive


In [ ]:
!pip install VnCoreNLP
!pip install transformers

     |████████████████████████████████| 2.6 MB 5.5 MB/s 
  Created wheel for VnCoreNLP: filename=vncorenlp-1.0.3-py3-none-any.whl size=2645951 sha256=a8e881218085a1b3f7177513362e42ddb277f23663a1f3f3e652a2eecad92b4b
  Stored in directory: /root/.cache/pip/wheels/0c/d8/f2/d28d97379b4f6479bf51247c8dfd57fa00932fa7a74b6aab29
Successfully built VnCoreNLP
     |████████████████████████████████| 3.5 MB 5.3 MB/s 
     |████████████████████████████████| 67 kB 5.3 MB/s 
     |████████████████████████████████| 6.8 MB 30.2 MB/s 
     |████████████████████████████████| 596 kB 43.4 MB/s 
     |████████████████████████████████| 895 kB 45.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [42]:
import re
import torch
import pandas as pd
import numpy as np
import tensorflow as tf
from vncorenlp import VnCoreNLP
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report
from transformers import AutoTokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_df = pd.read_csv(repo + "csv/train.csv", encoding='utf-16')
test_df = pd.read_csv(repo + "csv/test.csv", encoding='utf-16')

In [ ]:
X_train = train_df["text"]
y_train = train_df[["label"]]
X_test = test_df["text"]
y_test = test_df[["label"]]

In [ ]:
# Encode the labels
encoder = OneHotEncoder()
y_train_one_hot = encoder.fit_transform(y_train).toarray()
# It is possible to directly transform y_test since the label sets are the same
y_test_one_hot = encoder.transform(y_test).toarray()

y_train_one_hot

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [ ]:
print(y_train.shape, y_train_one_hot.shape)
print(y_test.shape, y_test_one_hot.shape)

(33759, 1) (33759, 10)
(50373, 1) (50373, 10)


### Preprocessing on a single sample

In [ ]:
# Sample text for testing
text = X_train[0]
print(text)

 Thành lập dự án POLICY phòng chống HIV/AIDS ở VN (NLĐ)- Quỹ hỗ trợ khẩn cấp về AIDS của Hoa Kỳ vừa thành lập dự án POLICY tại VN với cam kết hỗ trợ Chính phủ và nhân dân VN đối phó HIV/AIDS.Dự án có nhiệm vụ chính là cải thiện công tác phòng chống HIV/AIDS thông qua các lĩnh vực xây dựng chính sách, rà soát các văn bản pháp luật, xây dựng chiến lược quảng bá, xây dựng chương trình đào tạo về phòng chống HIV/AIDS, lên kế hoạch bố trí nguồn lực, huấn luyện và nghiên cứu về phương tiện truyền thông đại chúng, tổ chức các hoạt động nhằm giảm kỳ thị và phân biệt đối xử đối với người có HIV/AIDS... Theo TTXVN, dự án POLICY đặc biệt quan tâm đến công tác truyền thông phòng chống HIV/AIDS, coi đây là một biện pháp tích cực và hữu hiệu trong việc phòng chống có hiệu quả HIV/AIDS. Thời gian tới, dự án POLICY sẽ tiếp tục tổ chức các hoạt động nhằm nâng cao nhận thức cho những người có trách nhiệm với công tác chỉ đạo phòng chống HIV/AIDS.




In [ ]:
def text_normalization(text):
    text = text.lower().strip()
    return re.sub('[!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n]', ' ', text)

In [ ]:
text = text_normalization(text)
print(text)

thành lập dự án policy phòng chống hiv aids ở vn  nlđ   quỹ hỗ trợ khẩn cấp về aids của hoa kỳ vừa thành lập dự án policy tại vn với cam kết hỗ trợ chính phủ và nhân dân vn đối phó hiv aids dự án có nhiệm vụ chính là cải thiện công tác phòng chống hiv aids thông qua các lĩnh vực xây dựng chính sách  rà soát các văn bản pháp luật  xây dựng chiến lược quảng bá  xây dựng chương trình đào tạo về phòng chống hiv aids  lên kế hoạch bố trí nguồn lực  huấn luyện và nghiên cứu về phương tiện truyền thông đại chúng  tổ chức các hoạt động nhằm giảm kỳ thị và phân biệt đối xử đối với người có hiv aids    theo ttxvn  dự án policy đặc biệt quan tâm đến công tác truyền thông phòng chống hiv aids  coi đây là một biện pháp tích cực và hữu hiệu trong việc phòng chống có hiệu quả hiv aids  thời gian tới  dự án policy sẽ tiếp tục tổ chức các hoạt động nhằm nâng cao nhận thức cho những người có trách nhiệm với công tác chỉ đạo phòng chống hiv aids 


Word segmentation.

In [43]:
rdrsegmenter = VnCoreNLP(repo + "vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m')
sentences = rdrsegmenter.tokenize(text)
print(sentences)

[['thành_lập', 'dự_án', 'policy', 'phòng_chống', 'hiv', 'aids', 'ở', 'vn', 'nlđ', 'quỹ', 'hỗ_trợ', 'khẩn_cấp', 'về', 'aids', 'của', 'hoa_kỳ', 'vừa', 'thành_lập', 'dự_án', 'policy', 'tại', 'vn', 'với', 'cam_kết', 'hỗ_trợ', 'chính_phủ', 'và', 'nhân_dân', 'vn', 'đối_phó', 'hiv', 'aids', 'dự_án', 'có', 'nhiệm_vụ', 'chính', 'là', 'cải_thiện', 'công_tác', 'phòng_chống', 'hiv', 'aids', 'thông_qua', 'các', 'lĩnh_vực', 'xây_dựng', 'chính_sách', 'rà_soát', 'các', 'văn_bản', 'pháp_luật', 'xây_dựng', 'chiến_lược', 'quảng_bá', 'xây_dựng', 'chương_trình', 'đào_tạo', 'về', 'phòng_chống', 'hiv', 'aids', 'lên', 'kế_hoạch', 'bố_trí', 'nguồn_lực', 'huấn_luyện', 'và', 'nghiên_cứu', 'về', 'phương_tiện', 'truyền_thông', 'đại_chúng', 'tổ_chức', 'các', 'hoạt_động', 'nhằm', 'giảm', 'kỳ_thị', 'và', 'phân_biệt', 'đối_xử', 'đối_với', 'người', 'có', 'hiv', 'aids', 'theo', 'ttxvn', 'dự_án', 'policy', 'đặc_biệt', 'quan_tâm', 'đến', 'công_tác', 'truyền_thông', 'phòng_chống', 'hiv', 'aids', 'coi', 'đây', 'là', 'một', 

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/874k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.encoder['thành_lập']

763

In [ ]:
tokens = [tokenizer.encode(sentences[0])]
tokens = pad_sequences(tokens, maxlen=100, truncating="post", padding="post")
print(tokens[0])

[    0   763   169     3  2137     3     3    25 33756     3  1425   291
  2498    28     3     7     3   164   763   169     3    35 33756    15
  1253   291   926     6   603 33756  2971     3     3   169    10   527
   159     8  1238   247  2137     3     3   652     9   518   141   502
  1851     9   909   584   141   995  2524   141   270   643    28  2137
     3     3    72   421  1681  2684  2019     6   410    28   715  1062
  9847   116     9   132   272   197 12922     6  3043  4622   190    18
    10     3     3    63     3   169     3   234   511    30   247  1062
  2137     3     3   774]


In [47]:
def text_vectorizer(dataset, maxlen=None):
    dataset = dataset.tolist()
    dataset = map(lambda x: text_normalization(x), dataset)
    sentences = [rdrsegmenter.tokenize(x) for x in dataset]
    tokens = [tokenizer.encode(sentence[0]) for sentence in sentences]
    tokens = pad_sequences(tokens, maxlen=maxlen, truncating="post", padding="post")
    return tokens

In [66]:
X_train_indices = text_vectorizer(X_train)
maxlen = X_train_indices.shape[1]
X_train_indices.shape

(33759, 1000)

In [67]:
X_test_indices = text_vectorizer(X_test, maxlen=maxlen)
X_test_indices.shape

(50373, 1000)

In [74]:
def cnn_model(filters, input_dim, output_dim, dropout_rate=0.2, maxlen=200):
  
    # Channel 1D CNN
    inp = tf.keras.Input(shape=(maxlen,), dtype='int32')
    embed = tf.keras.layers.Embedding(input_dim, output_dim)(inp)
    conv1 = tf.keras.layers.Conv1D(filters=filters, kernel_size=2, activation='relu')(embed)
    pool1 = tf.keras.layers.GlobalMaxPool1D()(conv1)
    conv2 = tf.keras.layers.Conv1D(filters=filters, kernel_size=3, activation='relu')(embed)
    pool2 = tf.keras.layers.GlobalMaxPool1D()(conv2)
    conv3 = tf.keras.layers.Conv1D(filters=filters, kernel_size=4, activation='relu')(embed)
    pool3 = tf.keras.layers.GlobalMaxPool1D()(conv3)
    conv4 = tf.keras.layers.Conv1D(filters=filters, kernel_size=5, activation='relu')(embed)
    pool4 = tf.keras.layers.GlobalMaxPool1D()(conv4)
    concat = tf.concat([pool1, pool2, pool3, pool4], axis=1)
    drop1 = tf.keras.layers.Dropout(dropout_rate)(concat)
    dense1 = tf.keras.layers.Dense(filters * 2, activation='relu')(drop1)
    drop2 = tf.keras.layers.Dropout(dropout_rate)(dense1)
    dense2 = tf.keras.layers.Dense(filters, activation='relu')(drop2)
    drop3 = tf.keras.layers.Dropout(dropout_rate)(dense2)
    
    # Interpretation
    outputs = tf.keras.layers.Dense(10, activation='softmax')(drop3)
    model = tf.keras.Model(inputs=inp, outputs=outputs)
    
    # Compile
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [75]:
model = cnn_model(filters=32, input_dim=len(tokenizer.encoder), output_dim=100, maxlen=maxlen)
model.summary()

Model: "model_5"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 1000)]       0           []                               
                                                                                                  
 embedding_5 (Embedding)        (None, 1000, 100)    6400000     ['input_6[0][0]']                
                                                                                                  
 conv1d_15 (Conv1D)             (None, 999, 32)      6432        ['embedding_5[0][0]']            
                                                                                                  
 conv1d_16 (Conv1D)             (None, 998, 32)      9632        ['embedding_5[0][0]']            
                                                                                            

In [76]:
model.fit(
    X_train_indices,
    y_train_one_hot,
    epochs=20,
    batch_size=64
)

Epoch 1/20
528/528 [==============================] - 27s 47ms/step - loss: 1.1977 - accuracy: 0.6023
Epoch 2/20
528/528 [==============================] - 23s 44ms/step - loss: 0.5109 - accuracy: 0.8467
Epoch 3/20
528/528 [==============================] - 23s 44ms/step - loss: 0.3344 - accuracy: 0.9009
Epoch 4/20
528/528 [==============================] - 24s 45ms/step - loss: 0.2232 - accuracy: 0.9341
Epoch 5/20
528/528 [==============================] - 23s 44ms/step - loss: 0.1499 - accuracy: 0.9568
Epoch 6/20
528/528 [==============================] - 23s 44ms/step - loss: 0.1103 - accuracy: 0.9684
Epoch 7/20
528/528 [==============================] - 23s 44ms/step - loss: 0.0857 - accuracy: 0.9751
Epoch 8/20
528/528 [==============================] - 23s 44ms/step - loss: 0.0769 - accuracy: 0.9779
Epoch 9/20
528/528 [==============================] - 23s 44ms/step - loss: 0.0658 - accuracy: 0.9819
Epoch 10/20
528/528 [==============================] - 23s 44ms/step - loss: 0.058

In [77]:
y_pred_one_hot = model.predict(X_test_indices)

In [78]:
y_pred = encoder.inverse_transform(y_pred_one_hot)
print(y_pred[::5000])
print(y_test[::5000].to_numpy())

[['Chinh tri Xa hoi']
 ['Chinh tri Xa hoi']
 ['Kinh doanh']
 ['Chinh tri Xa hoi']
 ['Phap luat']
 ['Suc khoe']
 ['The gioi']
 ['The thao']
 ['Van hoa']
 ['Van hoa']
 ['Vi tinh']]
[['Chinh tri Xa hoi']
 ['Chinh tri Xa hoi']
 ['Khoa hoc']
 ['Kinh doanh']
 ['Phap luat']
 ['Suc khoe']
 ['The gioi']
 ['The thao']
 ['Van hoa']
 ['Van hoa']
 ['Vi tinh']]


In [79]:
print(classification_report(y_test, y_pred, digits=4))

                  precision    recall  f1-score   support

Chinh tri Xa hoi     0.7949    0.8669    0.8293      7567
        Doi song     0.6160    0.6586    0.6366      2036
        Khoa hoc     0.7647    0.7257    0.7447      2096
      Kinh doanh     0.8720    0.8614    0.8667      5276
       Phap luat     0.9125    0.8561    0.8834      3788
        Suc khoe     0.9269    0.8920    0.9091      5417
        The gioi     0.9520    0.8772    0.9131      6716
        The thao     0.9705    0.9787    0.9746      6667
         Van hoa     0.9148    0.9282    0.9215      6250
         Vi tinh     0.9011    0.9333    0.9169      4560

        accuracy                         0.8837     50373
       macro avg     0.8626    0.8578    0.8596     50373
    weighted avg     0.8862    0.8837    0.8843     50373



In [80]:
model.save('model')

INFO:tensorflow:Assets written to: model/assets


In [82]:
!zip -r model.zip model

  adding: model/ (stored 0%)
  adding: model/assets/ (stored 0%)
  adding: model/keras_metadata.pb (deflated 92%)
  adding: model/saved_model.pb (deflated 90%)
  adding: model/variables/ (stored 0%)
  adding: model/variables/variables.index (deflated 69%)
  adding: model/variables/variables.data-00000-of-00001 (deflated 48%)
